In [1]:
import re
import pandas as pd
import numpy as np
from datetime import datetime
from dataprep.clean import clean_text
from dataprep.clean import clean_headers
from nltk.tokenize import sent_tokenize

/usr/local/Caskroom/miniconda/base/envs/analytics/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
input_df = pd.read_csv("Google_reviews_Nakheel_Mar22-Mar23.csv")

In [3]:
input_df = clean_headers(input_df)
input_df.head()

Column Headers Cleaning Report:
	12 values cleaned (100.0%)


,source,content,sender_profile_image_url,created_time,permalink,sn_type_column,associated_cases,locations,date,ratings,insights,experience_score
0,Google My Business,NaN,https://lh3.googleusercontent.com/a-/ACB-R5R-D...,2023-03-01 13:16:28,NaN,GOOGLE_PLUS,NaN,The View,March 2023,5.0,0,NaN
1,Google My Business,NaN,https://lh3.googleusercontent.com/a-/ACB-R5T1U...,2023-03-01 10:06:59,NaN,GOOGLE_PLUS,NaN,The View,March 2023,5.0,0,NaN
2,Google My Business,(Translated by Google) A beautiful place for w...,https://lh3.googleusercontent.com/a-/AD5-WClJD...,2023-03-01 01:09:10,NaN,GOOGLE_PLUS,NaN,The Pointe,March 2023,5.0,0,NaN
3,Google My Business,Great place,https://lh3.googleusercontent.com/a-/AD5-WClb2...,2023-03-01 00:29:24,NaN,GOOGLE_PLUS,NaN,The View,March 2023,5.0,0,NaN
4,Google My Business,(Translated by Google) Dubai's best views\n\n(...,https://lh3.googleusercontent.com/a-/ACB-R5R1y...,2023-03-01 15:24:50,NaN,GOOGLE_PLUS,NaN,The View,March 2023,5.0,1,100.0


In [4]:
def empty_null(text:str) -> str:
  if str(text) == "":
    text = np.NaN
  return text

In [5]:
df = input_df[['content','date','locations','ratings']]
df['content'] = df['content'].astype(str)
df['content'] = df['content'].str.split('(Original)').str[0]

custom_pipeline = [
    {"operator": "fillna"},
    {"operator": "lowercase"},
    {"operator": "remove_digits"},
    {"operator": "remove_html"},
    {"operator": "remove_urls"},
    # {"operator": "remove_punctuation"},
    {"operator": "replace_punctuation", "parameters": {"value": ""}},
    {"operator": "remove_whitespace"},
    {"operator": empty_null}
]

df['raw_content'] = df.content
reviews_df = clean_text(df, "content", pipeline=custom_pipeline)
reviews_df.content = reviews_df.content.str.replace('translated by google','')
reviews_df.content = reviews_df.content.str.replace('translated by google','')
# reviews_df.drop_duplicates(inplace=True)

/var/folders/76/4d6cdpw57qdfw434ck3t8_3w0000gp/T/ipykernel_76362/2534648068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'] = df['content'].astype(str)
/var/folders/76/4d6cdpw57qdfw434ck3t8_3w0000gp/T/ipykernel_76362/2534648068.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'] = df['content'].str.split('(Original)').str[0]
/var/folders/76/4d6cdpw57qdfw434ck3t8_3w0000gp/T/ipykernel_76362/2534648068.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [6]:
reviews_df.ratings = reviews_df.ratings.astype(int)
reviews_df["content"].fillna("no feedback", inplace = True)
reviews_df = reviews_df.astype(str)
reviews_df= reviews_df.apply(lambda x: x.str.strip())
reviews_df.head()
reviews_df.to_csv("delete.csv",index=None)

In [7]:
reviews_df

,content,date,locations,ratings,raw_content
0,no feedback,March 2023,The View,5,nan
1,no feedback,March 2023,The View,5,nan
2,a beautiful place for walking and family holidays,March 2023,The Pointe,5,(Translated by Google) A beautiful place for w...
3,great place,March 2023,The View,5,Great place
4,dubais best views,March 2023,The View,5,(Translated by Google) Dubai's best views\n\n(
...,...,...,...,...,...
17014,de los mejores malls de dubái es de buen gusto...,March 2022,Nakheel Mall,5,De los mejores malls de Dubái. Es de buen gust...
17015,quite mall and best cafe shops,March 2022,Nakheel Mall,5,Quite mall and best cafe shops.
17016,good community mall with excellent stores amen...,March 2022,Circle Mall - JVC,5,Good community mall with excellent stores & am...
17017,no feedback,March 2022,Nakheel Mall,4,nan


In [11]:
reviews_df['content_api_input'] = \
" An anonymous customer gave a "\
  + reviews_df.ratings \
      + " star rating for '" + reviews_df.locations \
            + "' on " + reviews_df.date \
              + " and said " + reviews_df.content

In [17]:
reviews_df = reviews_df.sort_values(by=['locations'])

In [18]:
reviews_df.head(5)

,content,date,locations,ratings,raw_content,content_api_input
13299,no feedback,May 2022,Circle Mall - JVC,4,nan,An anonymous customer gave a 4 star rating fo...
13370,no feedback,May 2022,Circle Mall - JVC,5,nan,An anonymous customer gave a 5 star rating fo...
13046,quite a new mall lots of shops with variety of...,May 2022,Circle Mall - JVC,5,Quite a new mall. Lots of shops with variety o...,An anonymous customer gave a 5 star rating fo...
15519,no feedback,March 2022,Circle Mall - JVC,5,nan,An anonymous customer gave a 5 star rating fo...
13727,no feedback,May 2022,Circle Mall - JVC,4,nan,An anonymous customer gave a 4 star rating fo...


In [19]:
with open("data/google_reviews.txt", 'w') as f:
    for entry in reviews_df['content_api_input']:
        f.write(entry + "\n")

In [ ]:
Dubai-based Nakheel is a world-leading master developer whose innovative, landmark projects form an iconic portfolio of master communities and residential, retail, hospitality and leisure developments that are pivotal to realising Dubai’s vision. 
Nakheel’s waterfront projects, including the world-famous, award-winning Palm Jumeirah, have added more than 300 kilometres to Dubai’s original, 70km coastline, paving the way for the development of hundreds of seafront homes, resorts, hotels and attractions.
Nakheel’s master developments span 15,000 hectares, accommodate nearly 300,000 people and provide a growing range of destinations, attractions and facilities for Dubai’s citizens, residents and tourists. They include Palm Jumeirah, The World, Jumeirah Islands, Jumeirah Park, Jumeirah Village, Al Furjan, The Gardens, Discovery Gardens, Jebel Ali Village, Dragon City, Nad Al Sheba Villas, Warsan Village, International City and the upcoming Dubai Islands – a new destination for tourism, living and leisure at Dubai’s oldest, most traditional trading hub.
Nakheel Malls owns and operates a diverse range of world-class retail experiences with a distinct portfolio of iconic destinations, including large-scale shopping centres and retail Pavilions, across Dubai.
Nakheel’s existing retail developments (also known as Nakheel Malls) in Dubai include Ibn Battuta Mall, Dragon Mart, Nakheel Mall, Golden Mile Galleria, The Pointe, Club Vista Mare, Souk Marfa, Souk Warsan, The View at The Palm as well as six community retail centres, known as Nakheel Pavilions.
An iconic experience in the heart of Palm Jumeirah, Nakheel Mall offers over 300 shops, restaurants, entertainment outlets and services, including a 15-screen cinema complex and other attractions across five floors.
The food meets fashion retail destination offers an eclectic collection of brands and experiences. Nakheel Mall offers an enhanced shopping experience, through its major attractions such as the interconnected Palm Monorail, The Rooftop as well as The View that connects the mall to the most iconic views in Dubai.
The world’s largest themed shopping mall, Ibn Battuta Mall, is designed around the travel experiences of renowned Arabian explorer, Ibn Battuta. At Ibn Battuta Mall, shopping, dining and entertainment come together with history, culture, and the story of a remarkable adventure. The mall features six mesmerizing individual courts –India Court, Persia Court, Egypt Court, Tunisia Court, Andalusia Court and at the centre of it all is one of the most seaworthy vessels of its time – a Chinese Junk situated in China Court. It’s homage to the great traveller and his remarkable adventure.
The Pointe is an iconic waterfront dining and entertainment destination on Palm Jumeirah where you will be spoilt for choice when it comes to dining options. Spanning 1.4 million square feet with a mix of food, entertainment and lifestyle offerings.
The Pointe is home to a 1.5km promenade, a dine-in cinema by Reel Cinemas, a children’s play area, supermarket, gyms, beauty salons and unique gifting concepts, as well as a car park for 1,400 vehicles. The Pointe is also home to The Palm Fountain that holds the Guinness World Record for The World’s Largest Fountain.
Circle Mall, the newest community mall by master developer Nakheel, opened its doors in April 2021. Located at Jumeirah Village Circle, just off Hessa Street, the mall is a family-friendly destination for residents of JVC and nearby areas, with leading retailers, food and entertainment brands.
Circle Mall spans 532,000 sq ft and opens with a collection of brands including Spinneys supermarket, Nesto Hypermarket, Daiso, Medicina Pharmacy, Du, Etisalat and everything a community needs for convenient shopping close to home.
The rooftop, which will be home to the first community club in JVC, will feature state-of-the-art amenities, including an outdoor pool, two tennis courts and McCafferty’s, a new Irish restaurant concept which will complement a host of other coffee shops and dining options.
Palm West Beach is a walkable waterfront destination at Palm Jumeirah for all-day entertainment, with water sports, a range of premium hotels and restaurants, water sports, and the best Dubai skyline views. Palm West Beach is home to 300 illuminating palm trees and is a pet friendly destination that stretches across 1.6km of promenade and beachfront.
A premium waterfront dining and beach club facility, Club Vista Mare is one of the Palm Jumeirah’s most popular destinations for both residents and tourists. It spans almost 50,000 square feet at ground level, with an additional 30,000 square feet of residential space on the first floor. Upscale lifestyle and dining destinations serve a fabulous range of Middle Eastern, Greek, Spanish and other cuisines. Each restaurant has indoor and outdoor terrace seating, with some featuring space directly on the beach.
The View at The Palm is a stunning new public observation deck towering 240 metres above Palm Jumeirah, on the 52nd floor of The Palm Tower. The attraction is the only location in Dubai offering the unique experience of spectacular views of Palm Jumeirah, the Arabian Gulf and the Dubai skyline, along with an interactive experience about the making of Palm Jumeirah.
Visitors to The View will start their journey at the roof plaza of Nakheel Mall. From there, an elevator, complete with a floor-to-ceiling digital sea, sand and sky experience, will take them on the three-minute ascent to the observatory at the top of The Palm Tower, where they can marvel at the island beneath them and enjoy 360-degree views of Dubai. Visitors can take this experience to new heights at The Next Level, a stunning private viewing space, 250 metres above ground,  the highest vantage point on Palm Jumeirah.
Golden Mile Galleria is a retail and dining destination which offers doorstep convenience for thousands of Palm Jumeirah residents. Located beneath the Golden Mile residential complex and adjacent to the beautiful Al Ittihad Park which awaits you at the entrance, Golden Mile Galleria offers 400,000 square feet of retail space and 1,600 underground parking spaces.
Located in Dragon City, Dragon Mart is the largest Chinese retail trading hub outside mainland China. The 1.2 kilometre long dragon-shaped Dragon Mart is divided into seven zones with over 3,500 stores offering a wide variety of products.
Dragon Mart 2 is the retail and lifestyle extension of Dragon Mart and one of the most popular family shopping, dining and entertainment destinations in Dubai. The 2.1 million square feet extension offers 1.1 million square feet of leasable space and features 570 new shops, 520 kiosks, restaurants and entertainment outlets, as well as a 12-screen Novo Cinema complex, food court, Dragon Drive and separate terrace with indoor and outdoor dining options.
Nakheel Pavilions are dedicated retail centres designed to meet the needs of each of our communities. Each pavilion features a supermarket, convenience stores, restaurants and other amenities for on-the-doorstep shopping and recreation. The Pavilions are Al Furjan Pavilion and Badrah Pavilion.
Souk Al Marfa, the waterfront wholesale souk and marketplace at Deira Islands, features 400 stores across various categories. Designed as an indoor marketplace, the fully air-conditioned souk is a centralised, one-stop-shop, stretching 1.9 kilometres along the waterfront.
Souk Al Marfa brings together businesses from local and international backgrounds in the form of small kiosks, retail stores, showrooms, and street food markets. Shop for a range of items, including carpets, lights, spices, fashion, electronics and more, and explore the authentic Thai Market offering Thai food, handicraft, street food and clothing. It also features Mohideen supermarket and fast-food restaurants.
Souk Al Marfa provides affordable growth opportunities to traders across the country, with the potential to ship and import directly to its shops and pavilions, and promises a diverse, large-scale shopping experience for customers. When fully operational, Souk Al Marfa will be the UAE’s largest wholesale souk and waterfront destination.
Souk Warsan with 650,000 sq. ft. of retail space, the souk will be spread across eight buildings, each comprising two floors of retail with three-bedroom townhouses built above.
This documents contains customer reviews of Nakheel's retail developments and it was scraped from google maps reviews
The customer reviews include the rating, location, date and review.